In [1]:
# Importações principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configuração dos gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# Configurações do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


ModuleNotFoundError: No module named 'plotly'

In [ ]:
# Carregar dados da estratégia híbrida Open-Meteo
print("🌦️ Carregando dados atmosféricos...")

# Dados principais - Historical Forecast API (149 variáveis, 2022-2025)
try:
    df_forecast = pd.read_csv('../data/processed/openmeteo_historical_forecast.csv')
    df_forecast['datetime'] = pd.to_datetime(df_forecast['datetime'])
    df_forecast.set_index('datetime', inplace=True)
    print(f"✅ Historical Forecast: {df_forecast.shape[0]} registros, {df_forecast.shape[1]} variáveis")
except FileNotFoundError:
    print("❌ Arquivo Historical Forecast não encontrado. Execute primeiro o pipeline de dados")
    # Tentar carregar dados brutos disponíveis
    try:
        from pathlib import Path
        raw_files = list(Path("../data/raw/Open-Meteo Historical Forecast/").glob("*.csv"))
        if raw_files:
            print(f"📁 Encontrados {len(raw_files)} arquivos brutos. Carregando o primeiro...")
            df_forecast = pd.read_csv(raw_files[0])
            df_forecast['datetime'] = pd.to_datetime(df_forecast['datetime'])
            df_forecast.set_index('datetime', inplace=True)
            print(f"✅ Dados brutos carregados: {df_forecast.shape}")
        else:
            df_forecast = None
    except Exception as e:
        print(f"❌ Erro carregando dados brutos: {e}")
        df_forecast = None

# Dados temporais - Historical Weather API (25 variáveis, 2000-2024)
try:
    df_weather = pd.read_csv('../data/processed/openmeteo_historical_weather.csv')
    df_weather['datetime'] = pd.to_datetime(df_weather['datetime'])
    df_weather.set_index('datetime', inplace=True)
    print(f"✅ Historical Weather: {df_weather.shape[0]} registros, {df_weather.shape[1]} variáveis")
except FileNotFoundError:
    print("❌ Arquivo Historical Weather não encontrado")
    # Tentar carregar dados brutos
    try:
        from pathlib import Path
        raw_files = list(Path("../data/raw/Open-Meteo Historical Weather/").glob("*.csv"))
        if raw_files:
            print(f"📁 Encontrados {len(raw_files)} arquivos brutos. Carregando o primeiro...")
            df_weather = pd.read_csv(raw_files[0])
            df_weather['datetime'] = pd.to_datetime(df_weather['datetime'])
            df_weather.set_index('datetime', inplace=True)
            print(f"✅ Dados brutos carregados: {df_weather.shape}")
        else:
            df_weather = None
    except Exception as e:
        print(f"❌ Erro carregando dados brutos: {e}")
        df_weather = None

# Dados INMET para validação
try:
    df_inmet = pd.read_csv('../data/processed/dados_inmet_processados.csv')
    df_inmet['datetime'] = pd.to_datetime(df_inmet['datetime'])
    df_inmet.set_index('datetime', inplace=True)
    print(f"✅ INMET: {df_inmet.shape[0]} registros, {df_inmet.shape[1]} variáveis")
except FileNotFoundError:
    print("❌ Dados INMET processados não encontrados")
    # Tentar carregar dados brutos INMET
    try:
        from pathlib import Path
        raw_files = list(Path("../data/raw/INMET/").glob("*.csv"))
        if raw_files:
            print(f"📁 Encontrados {len(raw_files)} arquivos INMET brutos.")
            df_inmet = pd.read_csv(raw_files[0])
            if 'Data Medicao' in df_inmet.columns:
                df_inmet['datetime'] = pd.to_datetime(df_inmet['Data Medicao'])
                df_inmet.set_index('datetime', inplace=True)
                print(f"✅ Dados INMET brutos carregados: {df_inmet.shape}")
            else:
                print("⚠️ Formato INMET não reconhecido")
                df_inmet = None
        else:
            df_inmet = None
    except Exception as e:
        print(f"❌ Erro carregando dados INMET: {e}")
        df_inmet = None


In [ ]:
# Resumo da análise exploratória
print("📋 RESUMO DA ANÁLISE EXPLORATÓRIA")
print("=" * 50)

if df_forecast is not None:
    print(f"✅ Dados Open-Meteo Historical Forecast carregados com sucesso")
    print(f"   - Período: {df_forecast.index.min().strftime('%Y-%m-%d')} até {df_forecast.index.max().strftime('%Y-%m-%d')}")
    print(f"   - Registros: {len(df_forecast):,}")
    print(f"   - Variáveis: {df_forecast.shape[1]}")
    
    # Identificar níveis de pressão disponíveis
    pressure_levels = []
    for level in ['1000hPa', '850hPa', '700hPa', '500hPa', '300hPa', '200hPa']:
        if any(level in col for col in df_forecast.columns):
            pressure_levels.append(level)
    
    if pressure_levels:
        print(f"   - Níveis de pressão: {', '.join(pressure_levels)}")
        print(f"   - ⭐ Análise sinótica habilitada (850hPa para frentes, 500hPa para vórtices)")
    
    # Qualidade dos dados
    missing_pct = (df_forecast.isnull().sum().sum() / df_forecast.size) * 100
    print(f"   - Dados faltantes: {missing_pct:.2f}%")
    
    if missing_pct < 1:
        print(f"   - ✅ Excelente qualidade dos dados")
    elif missing_pct < 5:
        print(f"   - ✅ Boa qualidade dos dados")
    else:
        print(f"   - ⚠️ Atenção: verificar tratamento de dados faltantes")

print("\n🎯 PRÓXIMOS PASSOS:")
print("1. 🔧 Feature Engineering atmosférica")
print("2. 🧠 Treinamento do modelo LSTM híbrido") 
print("3. 📊 Avaliação de performance")
print("4. ✅ Validação cruzada temporal")

print("\n🚀 Estratégia híbrida pronta para implementação!")
print(f"   Target de precisão: >82% (vs ~70% modelos tradicionais)")
